In [102]:
# %%time
import shutil 
import json
from fastai.vision.all import *
from clearml import StorageManager
import numpy as np


def deterministic_random_split(list_of_str, p1, p2=None):
    """
    Deterministically return a int list of length len(list_of_str) that splits the list.
    The portion of 0's in the returned array should be p1 up to rounding to 3 decimal points.
    The split is performed by pseudo-randomly mapping the string to integers in the interval 0-1000
    and entries whose hash is below p1*1000 are labelled 0.
    p1 is the percentage of data that goes into the training set.
    If p2 is not set, 1-p1 goes to the validation set.
    If p2 is set, p2 goes to the test set, 1-p1-p2 goes to the validation set
    """

    def convert_string_to_value(string, p1, p2):
        hash_value = int(hashlib.sha1(string.encode("utf-8")).hexdigest(), 16) % 1000

        threshold1 = p1 * 1000
        threshold2 = 1000
        if p2 is not None:
            threshold2 = (p1 + 1 - p1 - p2) * 1000

        if hash_value < threshold1:
            # If hash value is less than first threshold it belongs to the training set
            return 0
        if hash_value < threshold2:
            # If hash value between threshold 1 and 2 it belongs to the validation set
            return 1
        # If hash value is greater than threshold 2
        return 2

    return [convert_string_to_value(s, p1, p2) for s in list_of_str]

def make_raw_data_and_eval_dataset():
    run_dataset_path = untar_data(URLs.PETS)
    items = get_image_files(run_dataset_path / "images")
    test_items = np.array(items)[
        np.array(deterministic_random_split([item.stem for item in items], p1=0.8)) > 0
    ]
    train_items = np.array(items)[
        np.array(deterministic_random_split([item.stem for item in items], p1=0.8)) == 0
    ]
    assert set.intersection(set(test_items), set(train_items)) == set()
    test_path = run_dataset_path / "test_images"
    test_path.mkdir(exist_ok=True)
    train_path = run_dataset_path / "train_images"
    train_path.mkdir(exist_ok=True)
    for item in test_items:
        shutil.copy(str(item), str(test_path))
    for item in train_items:
        shutil.copy(str(item), str(train_path))

    # upload raw files to gs
    manager = StorageManager()
    manager.upload_folder(
        train_path, "gs://clearml-evaluation/data/pets/raw_incoming_data"
    )
    manager.upload_folder(
        test_path, "gs://clearml-evaluation/data/pets/evaluation_data"
    )

    with open(run_dataset_path/"train_items.json",'w') as fp:
        json.dump([a.stem for a in train_items], fp)
    manager.upload_file(run_dataset_path/"train_items.json","gs://clearml-evaluation/data/pets/train_items.json" )
    print(test_items)
    with open(run_dataset_path/"evaluation_items.json",'w') as fp:
        json.dump([a.stem for a in test_items], fp)
    manager.upload_file(run_dataset_path/"evaluation_items.json","gs://clearml-evaluation/data/pets/evaluation_items.json" )
        
    
    # create an evaluation dataset
    eval_dataset = Dataset.create(
        dataset_name=f"pets_evaulation",
        dataset_project="lavi-testing",
        parent_datasets= None,
    )
    eval_dataset.add_files(test_path)
    eval_dataset.upload()
    eval_dataset.finalize()

#make_raw_data_and_eval_dataset()

[Path('/root/.fastai/data/oxford-iiit-pet/images/Russian_Blue_240.jpg')
 Path('/root/.fastai/data/oxford-iiit-pet/images/wheaten_terrier_38.jpg')
 Path('/root/.fastai/data/oxford-iiit-pet/images/havanese_23.jpg') ...
 Path('/root/.fastai/data/oxford-iiit-pet/images/german_shorthaired_44.jpg')
 Path('/root/.fastai/data/oxford-iiit-pet/images/keeshond_150.jpg')
 Path('/root/.fastai/data/oxford-iiit-pet/images/havanese_43.jpg')]


In [ ]:
import torch
from fastai.vision.all import (
    partial,
    Precision,
    Recall,
    get_image_files,
    using_attr,
    Datasets,
    vision_learner,
    L,
    RegexLabeller,
    PILImage,
    Resize,
    ToTensor,
    IntToFloatTensor,
    Categorize,
    aug_transforms,
    resnet34,
    accuracy,
    error_rate,
    SaveModelCallback,
    top_k_accuracy,
    load_learner,
    plt,
    pickle,
    json,
    Interpretation,
    ClassificationInterpretation,
)

# import timm
from fastai.callback.tensorboard import TensorBoardCallback

from clearml.automation.controller import PipelineDecorator
from clearml import TaskTypes, Task
from pathlib import Path


class TaskPaths:
    def __init__(self, task_name):
        from pathlib import Path
        root_path = Path("/data/clearml_evaluation") / task_name
        self.root_path = root_path
        self.tb_path = root_path / "tboard"
        self.tb_path.mkdir(parents=True, exist_ok=True)
        self.datasets_path = root_path / "datasets"
        self.datasets_path.mkdir(parents=True, exist_ok=True)
        self.models_path = root_path / "models"
        self.models_path.mkdir(parents=True, exist_ok=True)
        self.splits_path = root_path / "splits"
        self.splits_path.mkdir(parents=True, exist_ok=True)
        self.evaluations_path = root_path / "evaluations"
        self.evaluations_path.mkdir(parents=True, exist_ok=True)


def make_dataset(datasets_path):
    from fastai.vision.all import untar_data, URLs
    from pathlib import Path
    datasets_path = Path(datasets_path)

    # The next line fetches files to /data/clearml_evaluation/fastai_image_classification/datasets/oxford-iiit-pet/
    # or skips them if they are already thereS
    run_dataset_path = untar_data(URLs.PETS, data=datasets_path) / "images"
    return run_dataset_path
    #return "/the_dataset_path"


def make_data_splits(splits_path, run_dataset_path, run_id, seed=42):
    from fastai.vision.all import get_image_files, RandomSplitter
    splits_path = Path(splits_path)
    run_dataset_path = Path(run_dataset_path)

    items = get_image_files(run_dataset_path)[:500]
    train_idxs, rest = RandomSplitter(seed=42, valid_pct=0.6)(items)
    validation_idxs, test_idxs = RandomSplitter(seed=42, valid_pct=0.5)(rest)
    train_splits = train_idxs, validation_idxs
    run_splits_path = splits_path / run_id
    run_splits_path.mkdir(parents=True, exist_ok=True)
    for fname, idxs in zip(
        ("train_idxs.txt", "validation_idxs.txt", "test_idxs.txt"),
        (train_idxs, validation_idxs, test_idxs),
    ):
        with open(run_splits_path / fname, "w") as fp:
            fp.write("\n".join([str(idx) for idx in idxs]))
    return run_splits_path


def top_1_accuracy(inp, targ, axis=-1):
    return top_k_accuracy(inp, targ, k=1, axis=axis)


def top_2_accuracy(inp, targ, axis=-1):
    return top_k_accuracy(inp, targ, k=2, axis=axis)


def top_3_accuracy(inp, targ, axis=-1):
    return top_k_accuracy(inp, targ, k=3, axis=axis)


def make_image_transforms():
    return [PILImage.create, Resize(460), ToTensor(), IntToFloatTensor()]


def make_dls(
    run_dataset_path,
    run_splits_path,
):
    items = get_image_files(run_dataset_path)
    with open(run_splits_path / "train_idxs.txt", "r") as fp:
        train_idxs = L(fp.read().split("\n"))
    with open(run_splits_path / "validation_idxs.txt", "r") as fp:
        validation_idxs = L(fp.read().split("\n"))
    with open(run_splits_path / "test_idxs.txt", "r") as fp:
        evaluation_idxs = L(fp.read().split("\n"))
    splits = (train_idxs, validation_idxs)

    labeller = using_attr(RegexLabeller(pat=r"^(.*)_\d+.jpg$"), "name")
    tfms = [
        make_image_transforms(),
        [labeller, Categorize()],
    ]

    dsets = Datasets(items, tfms, splits=splits)
    dls = dsets.dataloaders(  # after_item=[ToTensor(), IntToFloatTensor()],
        batch_tfms=aug_transforms(size=224),
    )
    tst_dl = dls.test_dl(items[evaluation_idxs], with_labels=True)

    return dls, tst_dl

def make_learner(dls, task_model_path, backbone_name="resnet34"):
    # backbone_fn = pretrained_resnet_34 = partial(timm.create_model, pretrained=True)

    return vision_learner(
        dls,
        backbone_name,
        metrics=[
            accuracy,
            error_rate,
            precision_micro(),
            recall_micro(),
            #             roc_auc,
            top_1_accuracy,
            top_2_accuracy,
            top_3_accuracy,
        ],
        path=task_model_path,
        pretrained=True,
    )


def precision_micro():
    return Precision(average="micro")


def recall_micro():
    return Recall(average="micro")


def save_model(learn):
    # learn.export(learn.path/'learn.pkl')
    dummy_inp = torch.stack([a[0] for a in learn.dls.train_ds[:2]]).cuda()
    torch.jit.save(
        torch.jit.trace(learn.model.cuda(), dummy_inp), learn.path / "model_jit_cuda.pt"
    )
    torch.jit.save(
        torch.jit.trace(learn.model.cpu(), dummy_inp.cpu()),
        learn.path / "model_jit_cpu.pt",
    )
    learn.export("learner.pkl")

def train_image_classifier(
    run_dataset_path,
    run_splits_path,
    run_tb_path,
    models_path,
    run_id,
    tb_path,
    backbone_name,
):
    dls, test_dl = make_dls(run_dataset_path, run_splits_path)
    task_model_path = models_path / run_id
    task_model_path.mkdir(parents=True, exist_ok=True)
    learn = make_learner(dls, task_model_path, backbone_name)
    suggestions = learn.lr_find()
    plt.show()
    tb_callback = TensorBoardCallback(
        log_dir=run_tb_path, trace_model=False, log_preds=False
    )
    learn.fine_tune(
        2,
        suggestions.valley,
        cbs=[
            SaveModelCallback(every_epoch=False),
            tb_callback,
        ],
    )
    save_model(learn)  # with_opt=False

    print("sample validation results")
    learn.show_results()
    plt.show()
    run_learner_path = learn.path / "learner.pkl"
    return run_learner_path

def eval_model(
    run_learner_path, run_dataset_path, run_splits_path, run_id, evaluations_path
):
    learn = load_learner(run_learner_path, cpu=False)
    learn.model.to(device="cuda")
    dls, test_dl = make_dls(run_dataset_path, run_splits_path)
    learn.dls = dls
    test_dl = test_dl.to(device="cuda")
    preds, y_true, losses = learn.get_preds(inner=False, dl=test_dl, with_loss=True)

    run_eval_path = evaluations_path / run_id
    run_eval_path.mkdir(parents=True, exist_ok=True)
    eval_results = {
        "run_id": run_id,
        "run_learner_path": run_learner_path,
        "run_dataset_path": run_dataset_path,
        "run_splits_path": run_splits_path,
        "metrics": {
            "top_1_accuracy": top_1_accuracy(preds, y_true),
            "top_2_accuracy": top_2_accuracy(preds, y_true),
            "top_3_accuracy": top_3_accuracy(preds, y_true),
        },
    }
    print(eval_results["metrics"])

    interp = Interpretation(learn=learn, dl=test_dl, losses=losses)
    interp.plot_top_losses(9, figsize=(15, 10))
    plt.show()
    interp = ClassificationInterpretation.from_learner(learn, dl=test_dl)
    interp.plot_confusion_matrix(figsize=(10, 10))
    #     clearml_task.logger.report_matplotlib_figure(
    #         title="Manual Reporting - confusion matrix", series=run_id, iteration=0, figure=plt
    #     )

    plt.show()
    with open(run_eval_path / "preds.pkl", "wb") as fid:
        pickle.dump({"preds": preds, "y_true": y_true}, fid)

    with open(run_eval_path / "evaluation_results.json", "w") as fid:
        json.dump(eval_results, fid, default=str, indent=4)
    return run_eval_path

In [ ]:
#from clearml import Task
def fastai_image_classification_pipeline(run_id, task_paths, backbone_name="resnet34"):


    config = {"run_id": run_id}
    print("run_id:", run_id)

    clearml_task = Task.init(project_name="lavi_evaluation", task_name=run_id)
    config = clearml_task.connect_configuration(
        config, name="run_config"
    )

    config["task_paths"] = json.loads(json.dumps(vars(task_paths), default=str))
    config["backbone_name"] = backbone_name

    print(config)
    print("make dataset")
    run_dataset_path = make_dataset(datasets_path=task_paths.datasets_path)
    config["run_dataset_path"] = str(run_dataset_path)
    
    print("make splits")
    run_splits_path = make_data_splits(
        task_paths.splits_path, run_dataset_path, run_id, seed=42
    )
    config["run_splits_path"] = str(run_splits_path)

    run_tb_path = task_paths.tb_path / run_id
    config['run_tb_path'] = str(run_tb_path)
    print("train model")
    run_learner_path = train_image_classifier(
        run_dataset_path,
        run_splits_path,
        run_tb_path,
        task_paths.models_path,
        run_id,
        task_paths.tb_path,
        backbone_name,
    )
    config["run_learner_path"] = str(run_learner_path)

    print("evaluate model")
    run_eval_path = eval_model(
        run_learner_path,
        run_dataset_path,
        run_splits_path,
        run_id,
        task_paths.evaluations_path,
    )
    config["run_eval_path"] = str(run_eval_path)
    #clearml_task.close()

    print("pipeline complete")
#     return config

In [ ]:
from datetime import datetime
import json
task_name = "fastai_image_classification"
task_paths = TaskPaths(task_name)
run_id = f"{task_name}_{datetime.utcnow().strftime('%Y_%m_%dT%H_%M_%S.%f')[:-3]}"

run_result = fastai_image_classification_pipeline(run_id, task_paths)

In [ ]:
# def eval_model(
#     run_learner_path, run_dataset_path, run_splits_path, run_id, evaluations_path
# ):
#     learn = load_learner(run_learner_path, cpu=False)
#     learn.model.to(device="cuda")
#     dls, test_dl = make_dls(run_dataset_path, run_splits_path)
#     learn.dls = dls
#     test_dl = test_dl.to(device="cuda")
#     preds, y_true, losses = learn.get_preds(inner=False, dl=test_dl, with_loss=True)

#     task_eval_path = evaluations_path / run_id
#     task_eval_path.mkdir(parents=True, exist_ok=True)
#     eval_results = {
#         "run_id": run_id,
#         "run_learner_path": run_learner_path,
#         "run_dataset_path": run_dataset_path,
#         "run_splits_path": run_splits_path,
#         "metrics": {
#             "top_1_accuracy": top_1_accuracy(preds, y_true),
#             "top_2_accuracy": top_2_accuracy(preds, y_true),
#             "top_3_accuracy": top_3_accuracy(preds, y_true),
#         },
#     }
#     print(eval_results["metrics"])

#     interp = Interpretation(learn=learn, dl=test_dl, losses=losses)
#     interp.plot_top_losses(9, figsize=(15, 10))
#     plt.show()
#     interp = ClassificationInterpretation.from_learner(learn, dl=test_dl)
#     import pdb; pdb.trace()
#     interp.plot_confusion_matrix(figsize=(10, 10))
#     task.logger.report_matplotlib_figure(
#         title="Manual Reporting - confusion matrix", series=run_id, iteration=0, figure=plt
#     )

#     plt.show()
#     with open(task_eval_path / "preds.pkl", "wb") as fid:
#         pickle.dump({"preds": preds, "y_true": y_true}, fid)

#     with open(task_eval_path / "evaluation_results.json", "w") as fid:
#         json.dump(eval_results, fid, default=str, indent=4)
#     return task_eval_path

# eval_model(
#     Path(run_paths["run_learner_path"]),
#     Path(run_paths["run_dataset_path"]),
#     Path(run_paths["run_splits_path"]),
#     Path(run_paths["run_id"]),
#     Path(run_paths['task_paths']['evaluations_path'])
# )

In [ ]:
# pip install plotly

In [ ]:
# import plotly
# fig = plotly.express.imshow(
#         matrix,
#         labels={'x': "Predicted", 'y': "True"},
#         x=['0', '1'],
#         y=['0', '1'],
#         color_continuous_scale='blues',
#         text_auto=True,
#     )

In [ ]:
    from fastai.vision.all import untar_data, URLs
    from pathlib import Path
    datasets_path = Path("/tmp")
    run_dataset_path = str(untar_data(URLs.PETS, data=datasets_path) / "images")

In [ ]:
run_dataset_path

   3669   14676   84319


In [ ]:
ls /data/clearml_evaluation/fastai_image_classification/datasets/

In [ ]:
from clearml import Dataset
pets_dataset =Dataset.create(dataset_name="pets_01", dataset_project="lavi_evaluation", dataset_tags =['v1'],)

In [ ]:
#break dataset into chunks

import shutil
chunk_size = 500
image_files = get_image_files(run_dataset_path)
chunks = [image_files[x:x+chunk_size] for x in range(0, len(image_files), chunk_size)]
chunks_path = Path("/tmp/pets")
chunks_path.mkdir(parents=True, exist_ok=True)
for i_chunk, chunk in enumerate(chunks):
    chunk_path = chunks_path / f"part_{i_chunk:03}"
    chunk_path.mkdir(parents=True, exist_ok=True)
    for image_file in chunk:
        shutil.copy(image_file, chunk_path)

In [ ]:
i_chunk

In [1]:
from fastai.vision.all import *
from clearml import Dataset
prev_dataset_id = None
datasets = []
chunks_path = Path("/tmp/pets")

for i_chunk, chunk_path in enumerate(csorted(list(chunks_path.glob("*")))):
    print(chunk_path)
    curr_dataset = Dataset.create(
        dataset_name=f"pets_v2_{i_chunk:03}",
        dataset_project="lavi_evaluation",
        parent_datasets= None if prev_dataset_id is None else [prev_dataset_id],
    )
    curr_dataset.add_files(chunk_path)
    curr_dataset.upload()
    curr_dataset.finalize()
    datasets.append(curr_dataset)
    prev_dataset_id = curr_dataset.id

/tmp/pets/part_000
ClearML results page: https://app.clear.ml/projects/e0943804bc0943d39066cc0045cbdc3b/experiments/9f10569909c94eb8bf7cc21caa43bfdb/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/e0943804bc0943d39066cc0045cbdc3b/experiments/9f10569909c94eb8bf7cc21caa43bfdb
Generating SHA2 hash for 500 files


100% 500/500 [00:00<00:00, 10531.26it/s]

Hash generation completed


Uploading dataset changes (500 files compressed to 49.39 MiB) to gs://clearml-evaluation
File compression and upload completed: total size 49.39 MiB, 1 chunked stored (average size 49.39 MiB)
/tmp/pets/part_001
ClearML results page: https://app.clear.ml/projects/64cf1767ea43434b8ff1eb14fa0039e6/experiments/913eb65a0077400c88633751b0497e2f/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/64cf1767ea43434b8ff1eb14fa0039e6/experiments/913eb65a0077400c88633751b0497e2f
Generating SHA2 hash for 500 files


100% 500/500 [00:00<00:00, 13021.34it/s]

...

Hash generation completed


Uploading dataset changes (500 files compressed to 53.01 MiB) to gs://clearml-evaluation
File compression and upload completed: total size 53.01 MiB, 1 chunked stored (average size 53.01 MiB)
/tmp/pets/part_004
ClearML results page: https://app.clear.ml/projects/f0ef3a8bb3874e21ae4a32a5fd62bb76/experiments/016c4bf1395b4eb5b073db010803434d/output/log
ClearML dataset page: https://app.clear.ml/datasets/simple/f0ef3a8bb3874e21ae4a32a5fd62bb76/experiments/016c4bf1395b4eb5b073db010803434d
Generating SHA2 hash for 500 files


100% 500/500 [00:00<00:00, 13226.15it/s]

Hash generation completed


Uploading dataset changes (500 files compressed to 52.17 MiB) to gs://clearml-evaluation
File compression and upload completed: total size 52.17 MiB, 1 chunked stored (average size 52.17 MiB)
2022-07-06 21:36:35,762 - clearml.Task - INFO - Waiting to finish uploads
2022-07-06 21:36:36,533 - clearml.Task - INFO - Finished uploading


In [106]:
curr_dataset.project

'lavi_evaluation'

In [114]:
curr_dataset.list_files()

['Abyssinian_10.jpg',
 'Abyssinian_100.jpg',
 'Abyssinian_108.jpg',
 'Abyssinian_11.jpg',
 'Abyssinian_110.jpg',
 'Abyssinian_114.jpg',
 'Abyssinian_125.jpg',
 'Abyssinian_130.jpg',
 'Abyssinian_131.jpg',
 'Abyssinian_133.jpg',
 'Abyssinian_134.jpg',
 'Abyssinian_149.jpg',
 'Abyssinian_150.jpg',
 'Abyssinian_154.jpg',
 'Abyssinian_155.jpg',
 'Abyssinian_156.jpg',
 'Abyssinian_157.jpg',
 'Abyssinian_159.jpg',
 'Abyssinian_161.jpg',
 'Abyssinian_169.jpg',
 'Abyssinian_172.jpg',
 'Abyssinian_173.jpg',
 'Abyssinian_174.jpg',
 'Abyssinian_180.jpg',
 'Abyssinian_183.jpg',
 'Abyssinian_184.jpg',
 'Abyssinian_19.jpg',
 'Abyssinian_190.jpg',
 'Abyssinian_191.jpg',
 'Abyssinian_192.jpg',
 'Abyssinian_195.jpg',
 'Abyssinian_196.jpg',
 'Abyssinian_197.jpg',
 'Abyssinian_215.jpg',
 'Abyssinian_22.jpg',
 'Abyssinian_224.jpg',
 'Abyssinian_225.jpg',
 'Abyssinian_230.jpg',
 'Abyssinian_25.jpg',
 'Abyssinian_32.jpg',
 'Abyssinian_33.jpg',
 'Abyssinian_37.jpg',
 'Abyssinian_4.jpg',
 'Abyssinian_49.jpg',

In [116]:
def make_dls(
    clearml_dataset,
    splits,
):
    from fastai.vision.all import (
        Path,
        Categorize,
        get_image_files,
        using_attr,
        RegexLabeller,
        Datasets,
        aug_transforms,
    )

    dataset_path = Path(clearml_dataset.get_local_copy())
    items = get_image_files(dataset_path)

    labeller = using_attr(RegexLabeller(pat=r"^(.*)_\d+.jpg$"), "name")
    tfms = [
        make_image_transforms(),
        [labeller, Categorize()],
    ]

    dsets = Datasets(items, tfms, splits=splits)
    dls = dsets.dataloaders(  # after_item=[ToTensor(), IntToFloatTensor()],
        batch_tfms=aug_transforms(size=224),
    )

    return dls

In [133]:
from pipeline_functions import *
dls = make_dls(
    curr_dataset,
    [range(len(curr_dataset.list_files())), range(len(curr_dataset.list_files()))],
)

In [127]:
dls.train_ds

(#2500) [(TensorImage([[[0.7686, 0.7882, 0.8196,  ..., 0.7255, 0.7373, 0.7451],
         [0.7804, 0.7961, 0.8196,  ..., 0.7255, 0.7373, 0.7451],
         [0.8039, 0.8118, 0.8235,  ..., 0.7255, 0.7412, 0.7490],
         ...,
         [0.8471, 0.8392, 0.8314,  ..., 0.5529, 0.5020, 0.4706],
         [0.8510, 0.8353, 0.8118,  ..., 0.5882, 0.5373, 0.5059],
         [0.8549, 0.8353, 0.8000,  ..., 0.6078, 0.5569, 0.5255]],

        [[0.7373, 0.7569, 0.7922,  ..., 0.7137, 0.7255, 0.7333],
         [0.7490, 0.7647, 0.7922,  ..., 0.7098, 0.7255, 0.7333],
         [0.7725, 0.7804, 0.7961,  ..., 0.7059, 0.7216, 0.7294],
         ...,
         [0.7686, 0.7608, 0.7490,  ..., 0.5412, 0.4941, 0.4627],
         [0.7725, 0.7569, 0.7294,  ..., 0.5686, 0.5216, 0.4902],
         [0.7765, 0.7529, 0.7176,  ..., 0.5843, 0.5412, 0.5098]],

        [[0.6627, 0.6824, 0.7176,  ..., 0.6392, 0.6510, 0.6588],
         [0.6745, 0.6902, 0.7176,  ..., 0.6353, 0.6510, 0.6588],
         [0.6980, 0.7059, 0.7216,  ..., 0.6

In [128]:
dls.path

Path('.')

In [136]:

test_dl = dls.test_dl(dls.items, with_labels=True)

In [140]:
L(test_dl.items)

(#2500) [Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/Egyptian_Mau_176.jpg'),Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/english_cocker_spaniel_10.jpg'),Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/Russian_Blue_240.jpg'),Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/wheaten_terrier_38.jpg'),Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/havanese_23.jpg'),Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/Maine_Coon_158.jpg'),Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/Abyssinian_22.jpg'),Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/Siamese_238.jpg'),Path('/root/.clearml/cache/storage_manager/datasets/ds_016c4bf1395b4eb5b073db010803434d/chihuahua_63.jpg'),